<a href="https://colab.research.google.com/github/ashfaq-khan14/AI-Data-Science-Conversational-Tutor/blob/main/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pypdf
!pip install langchain
!pip install google-generativeai
!pip install langchain_google_genai
!pip install chromadb

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"/content/Preprint.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='Preprint. Under review.\nLeave No Context Behind:\nEfficient Infinite Context Transformers with Infini-attention\nTsendsuren Munkhdalai, Manaal Faruqui and Siddharth Gopal\nGoogle\ntsendsuren@google.com\nAbstract\nThis work introduces an efficient method to scale Transformer-based Large\nLanguage Models (LLMs) to infinitely long inputs with bounded memory\nand computation. A key component in our proposed approach is a new at-\ntention technique dubbed Infini-attention. The Infini-attention incorporates\na compressive memory into the vanilla attention mechanism and builds\nin both masked local attention and long-term linear attention mechanisms\nin a single Transformer block. We demonstrate the effectiveness of our\napproach on long-context language modeling benchmarks, 1M sequence\nlength passkey context block retrieval and 500K length book summarization\ntasks with 1B and 8B LLMs. Our approach introduces minimal bounded\nmemory parameters and enables fast strea

In [3]:
page = "".join([p.page_content for p in pages])


In [7]:
import google.generativeai as genai
f = open("/content/Rag_geminiapi_key.txt")
key = f.read()

genai.configure(api_key=key)

In [8]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=500, chunk_overlap=100)

chunks = text_splitter.split_documents(pages)

print(len(chunks))

print(type(chunks[0]))

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [15]:
embedding_model= GoogleGenerativeAIEmbeddings(google_api_key=key, model="models/embedding-001")

In [ ]:
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db_")
db.persist()

In [ ]:
db_connection = Chroma(persist_directory="./chroma_db_", embedding_function=embedding_model)

In [ ]:
retriever = db_connection.as_retriever(search_kwargs={"k":5})
print(type(retriever))

In [12]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')

chat = model.start_chat(history=[])

chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-pro-latest',
        generation_config={},
        safety_settings={},
        tools=None,
        system_instruction=None,
    ),
    history=[]
)

In [13]:
user_input = page+ " previous text is the content of the research paper Leave No Context Behind .based on that answer the question" + "What is the most important topic that covered in the research paper named Leave No Context Behind"

response = chat.send_message(user_input)

print(response.text)

## Leave No Context Behind: Key Topic 

The research paper "Leave No Context Behind" primarily focuses on **scaling Transformer-based Large Language Models (LLMs) to process infinitely long input sequences while maintaining efficiency**. This is achieved through a novel attention mechanism called **Infini-attention**.

Here's a breakdown of the key elements:

* **Problem:** Standard Transformer models struggle with long sequences due to quadratic complexity in memory and computation. This limits their ability to handle extensive contexts effectively.
* **Proposed Solution: Infini-attention:** This new attention mechanism incorporates a compressive memory into the traditional attention framework. It combines:
    * **Masked local attention:** Focuses on the current input segment for detailed context.
    * **Long-term linear attention:** Retrieves information from the compressed memory of previous segments, enabling access to the entire context history. 
* **Benefits:**
    * **Bounded 